In [1]:
from helper import DataPreprocessing
from ipywidgets import interact, IntSlider
import librosa
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def peak_frequency (spec,freq):
    peak_freqs = np.zeros(spec.shape[1])
    for i in range(spec.shape[1]):
        idx = np.argmax(spec[:, i])
        peak_freqs[i] = freq[idx] 
    return peak_freqs

In [3]:
@interact(kind = ["train","full","test"], w_stft = IntSlider(min=100,max=25600,step=1,value=2560),hop = IntSlider(min=10,max=2560,step=1,value=256), w_name =  ["hann","boxcar","triang"])
def choose_kind_data(kind, w_stft,hop,w_name):

    #load data
    data = DataPreprocessing(kind)
    
    #load method
    mt = "Short Time Fourier Transform"
    method = "STFT"
    
    #load parameter:
    w_size = data.window_size_stft
    sr = data.sr
    
    if kind in ["train", "full"]:
        if kind == "train":
            bearing_number =  [11,12,21,22,31,32]
        else:
            bearing_number =  [13,14,15,16,17,23,24,25,26,27,33]
        @interact(bearing = bearing_number)
        def load_data_train(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            eol = data.eol(bearing)
            time_stamp = np.linspace(0,eol,N)
            stand = data.stand(bearing)
            hor_acc = stand[0]
            ver_acc = stand[1]

            @interact(label = IntSlider(min=0,max=eol-90,step=10))
            def visualize_STFT (label):

                #scale data with label and step
                s_step = N - label*sr//100 - w_size 
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((eol-label)-90,(eol-label)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                print("hor_acc_w:", hor_acc_w)
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()
                
                #visualize
                fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {}s, {} samples\nMethod {}".format(kind,bearing,eol,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal vibration, time step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,"darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal vibration window, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                
                #axes 3:
                stft_hor = librosa.stft(y=hor_acc_w,window=w_name,n_fft=w_stft,hop_length=hop)
                freq = librosa.fft_frequencies(sr=sr, n_fft=w_stft)
                spec_hor = np.abs(stft_hor)
                spec_hor = librosa.power_to_db(spec_hor)
                image_stft_hor = ax3.imshow(spec_hor, extent=[time_stamp_w_real.min(),time_stamp_w_real.max(), freq.max(), freq.min()], interpolation="none", aspect="auto")
                plt.colorbar(image_stft_hor,ax=ax3)
                ax3.set_title("{}, size {}".format(method,stft_hor.shape))
                ax3.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax3.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax3.set_ylabel("Frequency Domain")
                ax3.set_xlabel("Time domain")

                #axes 4:
                peak_hor = peak_frequency(spec_hor,freq)
                ax4.plot(np.linspace((eol-label)-100,(eol-label)+0.1,spec_hor.shape[1]),peak_hor)
                ax4.set_xlabel("Time domain")
                ax4.set_ylabel("Peak Frequency")
                ax4.set_title("Peak Frequency")

                #axes 5:
                ax5.plot(time_stamp,ver_acc)
                ax5.plot(time_stamp_w,ver_acc_w)
                ax5.legend(["vertikal","window"],loc="upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Vertikal vibration,time step {}".format(int(t_step)))
                
                #axes 6:
                ax6.plot(time_stamp_w_real,ver_acc_w,"darkorange")
                ax6.legend(["vertikal window"],loc = "upper left")
                ax6.set_ylabel("Normalized value")
                ax6.set_xlabel("Timestamp window")
                ax6.set_title("Horizontal vibration window label {}".format(label))
                ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 7:
                stft_ver = librosa.stft(y=ver_acc_w,window=w_name,n_fft=w_stft,hop_length=hop)
                freq = librosa.fft_frequencies(sr=sr, n_fft=w_stft)
                spec_ver = np.abs(stft_ver)
                spec_ver = librosa.power_to_db(spec_ver)
                image_stft_ver = ax7.imshow(spec_ver, extent=[time_stamp_w_real.min(),time_stamp_w_real.max(), freq.max(), freq.min()], interpolation="none", aspect="auto")
                plt.colorbar(image_stft_ver,ax=ax7)
                ax7.set_title("{}, size {}".format(method,stft_ver.shape))
                ax7.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax7.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax7.set_ylabel("Frequency domain")
                ax7.set_xlabel("Time domain") 

                #axes 8:
                peak_ver = peak_frequency(spec_ver,freq)
                ax8.set_xlabel("Time domain")
                ax8.set_ylabel("Peak Frequency")
                ax8.set_title("Peak Frequency")
                ax8.plot(np.linspace((eol-label)-100,(eol-label)+0.1,spec_ver.shape[1]),peak_ver) 

                print("Done")

    elif kind == "test":
        @interact(bearing = [13,14,15,16,17,23,24,25,26,27,33])
        def load_data_test(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            last_cycle = data.last_cycle(bearing)
            eol = data.eol(bearing)
            rul = data.rul(bearing)
            time_stamp = np.linspace(0,last_cycle,N)
            stand = data.stand(bearing)
            w_size = data.window_size_stft
            hor_acc = stand[0]
            ver_acc = stand[1]
            sr = data.sr

            @interact(label = IntSlider(min=rul,max=rul+last_cycle-90,step=10))
            def visualize_STFT (label):

                #scale data with label and step
                s_step = N - (label-rul)*sr//100 - w_size 
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((last_cycle-label+rul)-90,(last_cycle-label+rul)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3, ax4),(ax5, ax6, ax7, ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {} s, RUL {} s, {} samples\nMethod {}".format(kind,bearing,eol,rul,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal vibration, step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,c="darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal vibration window, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                
                #axes 3:
                stft_hor = librosa.stft(y=hor_acc_w,window=w_name,n_fft=w_stft,hop_length=hop)
                freq = librosa.fft_frequencies(sr=sr, n_fft=w_stft)
                spec_hor = np.abs(stft_hor)
                spec_hor = librosa.power_to_db(spec_hor)
                image_stft_hor = ax3.imshow(spec_hor, extent=[time_stamp_w_real.min(),time_stamp_w_real.max(), freq.max(), freq.min()], interpolation="none", aspect="auto")
                plt.colorbar(image_stft_hor,ax=ax3)
                ax3.set_title("{}, size {}".format(method,stft_hor.shape))
                ax3.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax3.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax3.set_ylabel("Frequency domain")
                ax3.set_xlabel("Time domain")

                #axes 4:
                peak_hor = peak_frequency(spec_hor,freq)
                ax4.plot(np.linspace((last_cycle-label+rul)-100,(last_cycle-label+rul)+0.1,spec_hor.shape[1]),peak_hor)
                ax4.set_xlabel("Time domain")
                ax4.set_ylabel("Peak Frequency")
                ax4.set_title("Peak Frequency")
                
                #axes 5:
                ax5.plot(time_stamp,ver_acc)
                ax5.plot(time_stamp_w,ver_acc_w)
                ax5.legend(["vertikal","window"],loc="upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Vertikal vibration, step {}".format(int(t_step)))
                
                #axes 6:
                ax6.plot(time_stamp_w_real,ver_acc_w,c="darkorange")
                ax6.legend(["vertikal window"],loc = "upper left")
                ax6.set_xlabel("Timestamp")
                ax6.set_ylabel("Normalized value")
                ax6.set_title("Horizontal vibration window, label {}".format(label))
                ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 7:
                stft_ver = librosa.stft(y=ver_acc_w,window=w_name,n_fft=w_stft,hop_length=hop)
                freq = librosa.fft_frequencies(sr=sr, n_fft=w_stft)
                spec_ver = np.abs(stft_ver)
                spec_ver = librosa.power_to_db(spec_ver)
                image_stft_ver = ax7.imshow(spec_ver, extent=[time_stamp_w_real.min(),time_stamp_w_real.max(), freq.max(), freq.min()], interpolation="none", aspect="auto")
                plt.colorbar(image_stft_ver,ax=ax7)
                ax7.set_title("{}, size {}".format(method,stft_hor.shape))
                ax7.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax7.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax7.set_ylabel("Frequency domain")
                ax7.set_xlabel("Time domain")

                #axes 8:
                peak_ver = peak_frequency(spec_ver,freq)
                ax8.set_xlabel("Time domain")
                ax8.set_ylabel("Peak Frequency")
                ax8.set_title("Peak Frequency")
                ax8.plot(np.linspace((last_cycle-label+rul)-100,(last_cycle-label+rul)+0.1,spec_ver.shape[1]),peak_ver)

                print("Done")
    

interactive(children=(Dropdown(description='kind', options=('train', 'full', 'test'), value='train'), IntSlide…